# VacationPy

In [2]:
#import dependencies 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [3]:
cities_path = 'outputs/cleaned_city_data.csv'

city_df = pd.read_csv(cities_path)

city_df_cleaned = city_df.dropna()
city_df_cleaned

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Grindavik,63.8424,-22.4338,52.27,71,75,9.22,IS,1660414090
1,Kapaa,22.0752,-159.3190,78.78,82,40,10.36,US,1660414092
2,Yeppoon,-23.1333,150.7333,56.98,89,42,4.38,AU,1660413804
3,George Town,5.4112,100.3354,80.89,82,20,3.44,MY,1660413605
4,Jamestown,42.0970,-79.2353,79.02,35,20,10.36,US,1660413741
...,...,...,...,...,...,...,...,...,...
561,Vanderhoof,54.0166,-124.0198,67.12,69,99,7.47,CA,1660415103
562,Cotonou,6.3654,2.4183,76.98,88,20,12.66,BJ,1660415403
563,Abapó,-18.8333,-63.4667,84.97,29,41,6.55,BO,1660415406
564,Gallup,35.5281,-108.7426,81.01,37,0,9.22,US,1660415170


## Humidity Heatmap

In [4]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [5]:
# Using lat and lng as locations
locations = city_df_cleaned[['Lat','Lng']]

humidity = city_df_cleaned['Humidity'].astype(float)

In [6]:
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer (locations, weights = humidity,
                                    dissipating = False, max_intensity = 100, point_radius = 2)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## Screenshot of humidity heatmap is in images directory

## Create new DataFrame fitting weather requirements

In [7]:
# narrow down cities to fit weather conditions

# zero clouds
cloud_df = city_df_cleaned[city_df_cleaned['Cloudiness'] == 0]

# max temp lower than 80 degrees, but higher than 70
temp_df = cloud_df[(cloud_df['Max Temp'] > 70) & (cloud_df['Max Temp'] < 80)]

# wind speed less than 10 mph
wind_df = temp_df[temp_df['Wind Speed'] < 10]

# drop rows with null values
wind_df = wind_df.dropna()
wind_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,Albany,42.6001,-73.9662,77.23,46,0,6.40,US,1660413599
68,Miandrivazo,-19.5167,45.4667,71.89,59,0,3.83,MG,1660414239
81,Bethel,41.3712,-73.4140,78.58,52,0,1.01,US,1660413716
89,Tula,54.2044,37.6111,73.60,57,0,6.62,RU,1660414285
197,Wahiawā,21.5028,-158.0236,77.32,72,0,0.00,US,1660414539
306,Revelstoke,50.9832,-118.2023,71.62,68,0,9.22,CA,1660414589
311,Mocuba,-16.8392,36.9856,70.43,61,0,6.06,MZ,1660414809
426,Zharkent,44.1628,80.0000,70.86,32,0,3.91,KZ,1660415081


## Hotel Map

In [10]:
# new DataFrame for hotel info
hotel_df = wind_df

# add column 'Hotel Name' to DataFrame
hotel_df['Hotel Name'] = ''

# set parameters to search for hotels within 5000 meters
params = {
    'radius': 5000,
    'types': 'hotel',
    'keyword': 'hotels',
    'key': g_key
}

# loop through hotel_df to find hotels
for index, row in hotel_df.iterrows():
    
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat},{lng}'
    
    hotel_data = requests.get(base_url, params=params).json()
    
    try:
        
        hotel_df.loc[index, 'Hotel Name'] = hotel_data['results'][0]['name']
    except:
        print('Issues with finding a hotel at', row['City'])
        hotel_df.loc[index, 'Hotel Name'] = 'NA'

hotel_df

Issues with finding a hotel at Albany


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
9,Albany,42.6001,-73.9662,77.23,46,0,6.40,US,1660413599,NA
68,Miandrivazo,-19.5167,45.4667,71.89,59,0,3.83,MG,1660414239,EDEN DE LA TSIRIBIHINA
81,Bethel,41.3712,-73.4140,78.58,52,0,1.01,US,1660413716,Hampton Inn Danbury
89,Tula,54.2044,37.6111,73.60,57,0,6.62,RU,1660414285,AZIMUT Сити Отель Тула
197,Wahiawā,21.5028,-158.0236,77.32,72,0,0.00,US,1660414539,The Inn At Schofield
306,Revelstoke,50.9832,-118.2023,71.62,68,0,9.22,CA,1660414589,Sutton Place Hotel Revelstoke Mountain Resort
311,Mocuba,-16.8392,36.9856,70.43,61,0,6.06,MZ,1660414809,Sāo Cristóvão
426,Zharkent,44.1628,80.0000,70.86,32,0,3.91,KZ,1660415081,гостиница Satti


In [12]:
hotel_df = hotel_df[hotel_df['Hotel Name'] != 'NA']
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
68,Miandrivazo,-19.5167,45.4667,71.89,59,0,3.83,MG,1660414239,EDEN DE LA TSIRIBIHINA
81,Bethel,41.3712,-73.4140,78.58,52,0,1.01,US,1660413716,Hampton Inn Danbury
89,Tula,54.2044,37.6111,73.60,57,0,6.62,RU,1660414285,AZIMUT Сити Отель Тула
197,Wahiawā,21.5028,-158.0236,77.32,72,0,0.00,US,1660414539,The Inn At Schofield
306,Revelstoke,50.9832,-118.2023,71.62,68,0,9.22,CA,1660414589,Sutton Place Hotel Revelstoke Mountain Resort
311,Mocuba,-16.8392,36.9856,70.43,61,0,6.06,MZ,1660414809,Sāo Cristóvão
426,Zharkent,44.1628,80.0000,70.86,32,0,3.91,KZ,1660415081,гостиница Satti


In [13]:
len(hotel_df[['Lat','Lng']])

7

In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# check len of hotel_info
len(hotel_info)

7

In [17]:
# add marker layer on top of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# display figure
fig

Figure(layout=FigureLayout(height='420px'))

## Screenshot of hotel map is in images directory